In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv


In [2]:
import numpy as np
import tensorflow as tf
from sklearn.decomposition import PCA

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
!nvidia-smi -L

/bin/bash: nvidia-smi: command not found


In [4]:
def print_tuple(obs):
    print("\033[1mprompt_question: \033[0m", obs.prompt_question)
    print()
    print("\033[1mprompt_title: \033[0m", obs.prompt_title)
    print()
    print("\033[1mprompt_text: \033[0m")
    print(obs.prompt_text)
    print()
    print("\033[1msummary: \033[0m")
    print(obs.text)
    print()
    print("\033[1mWording: \033[0m")
    print(obs.wording)
    print()
    print("\033[1mContent: \033[0m")
    print(obs.content)
    

In [5]:
#Filepath
FILE_PATH='/kaggle/input/commonlit-evaluate-student-summaries'
#train set
pmt_train=pd.read_csv(os.path.join(FILE_PATH,'prompts_train.csv'))
sum_train=pd.read_csv(os.path.join(FILE_PATH,'summaries_train.csv'))
df_train=pmt_train.merge(sum_train,how='inner',on='prompt_id')

In [6]:
#df_train
tuple1=df_train.iloc[0]
print_tuple(tuple1)

prompt_question:  Summarize at least 3 elements of an ideal tragedy, as described by Aristotle.

prompt_title:  On Tragedy

prompt_text: 
Chapter 13 
As the sequel to what has already been said, we must proceed to consider what the poet should aim at, and what he should avoid, in constructing his plots; and by what means the specific effect of Tragedy will be produced. 
A perfect tragedy should, as we have seen, be arranged not on the simple but on the complex plan. It should, moreover, imitate actions which excite pity and fear, this being the distinctive mark of tragic imitation. It follows plainly, in the first place, that the change of fortune presented must not be the spectacle of a virtuous man brought from prosperity to adversity: for this moves neither pity nor fear; it merely shocks us. Nor, again, that of a bad man passing from adversity to prosperity: for nothing can be more alien to the spirit of Tragedy; it possesses no single tragic quality; it neither satisfies the moral

In [7]:
#we actually dont want student_id or prompt_id for training hence dropping those.
df_train.drop(['student_id','prompt_id'],axis=1,inplace=True)

In [8]:
from sklearn.model_selection import train_test_split 
X = df_train[['text']]
y = df_train[['wording']]
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
#PREPROCESSING
from tensorflow.keras.layers import TextVectorization
max_len=round(sum([len(i.split()) for i in X['text']])/len(X))
#max_len will be length of each sequence it is the average no of words in each row.
text_vectorizer=TextVectorization(
    max_tokens=20000,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,#Creating groups of tokens
    output_mode='int',
    output_sequence_length=max_len,#how long each sequence of tokens should be.
    pad_to_max_tokens=True#Adding zeros at the end to reach the sequence length
    
)
#Fitting the text vectorizer to training data
text_vectorizer.adapt(X)
#No of unique words the vectorizer found
words_vocab=text_vectorizer.get_vocabulary()
print(len(words_vocab))
most_common=words_vocab[:5]
least_common=words_vocab[-5:]

14172


In [10]:
X

,text
0,1 element of an ideal tragedy is that it shoul...
1,The three elements of an ideal tragedy are: H...
2,Aristotle states that an ideal tragedy should ...
3,One element of an Ideal tragedy is having a co...
4,The 3 ideal of tragedy is how complex you need...
...,...
7160,"In paragraph two, they would use pickle meat a..."
7161,"in the first paragraph it says ""either can it..."
7162,They would have piles of filthy meat on the fl...
7163,They used all sorts of chemical concoctions to...


In [11]:
text_vectorizer(X.iloc[0])
#Shape is 1X75 means each of the summary is converted into a 75 tokens

<tf.Tensor: shape=(1, 75), dtype=int64, numpy=
array([[267, 115,   5,  28,  60,  22,  16,  10,   7,  19,  14, 263,  24,
          6,  88, 147, 167, 115,   5,  28,  60,  22,  16,  10,   7,  19,
        118,  40,  61, 145, 265,   2, 315, 115,   5,  28,  60,  22,  16,
         10,   7,  19,  40,   6, 442, 775,  70,   3,  28, 620, 595,  23,
        648,  33,   3,  34,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])>

In [12]:
from tensorflow.keras import layers
embeddings=layers.Embedding(
    input_dim=len(words_vocab),#length of vocabulary
    output_dim=128,
    embeddings_initializer='uniform',
    input_length=max_len,#The output of vectorizer is having fixed length
)

In [13]:
embeddings(text_vectorizer(X.iloc[0]))#Each of token  is converted into embedding of 128 dimensions

<tf.Tensor: shape=(1, 75, 128), dtype=float32, numpy=
array([[[-0.01401095, -0.02194986,  0.00041576, ...,  0.01193162,
          0.02613067, -0.02194109],
        [ 0.0478672 ,  0.0225344 ,  0.00594176, ..., -0.01226784,
         -0.00302178, -0.0029181 ],
        [-0.00628922, -0.02616367,  0.03149298, ..., -0.01546922,
         -0.00411373,  0.00740786],
        ...,
        [ 0.02939013,  0.03106311, -0.00770146, ..., -0.02791371,
          0.03368649, -0.03706723],
        [ 0.02939013,  0.03106311, -0.00770146, ..., -0.02791371,
          0.03368649, -0.03706723],
        [ 0.02939013,  0.03106311, -0.00770146, ..., -0.02791371,
          0.03368649, -0.03706723]]], dtype=float32)>

In [14]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype=tf.string)#shape is 1 since we will be feeding one string at a time.
x=text_vectorizer(inputs)#turning strings into vectors
y=embeddings(x)
z=layers.GlobalAveragePooling1D()(y)#condensing feature vector of each token to a single one
outputs=layers.Dense(units=1)(z)#no of hidden layers will be 1.Sigmoid activation since we need binary classification
model01=tf.keras.Model(inputs,outputs,name="model01")

In [15]:
model01.summary()

Model: "model01"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 75)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 75, 128)           1814016   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,814,145
Trainable params: 1,814,145
Non-tra

In [16]:
model01.compile(loss="mean_squared_error",optimizer=tf.keras.optimizers.Adam(),metrics=['mae'])

In [17]:
model01.fit(X_train, y_train, epochs=20, validation_data=(X_eval, y_eval))

Epoch 1/20
180/180 [==============================] - 3s 15ms/step - loss: 0.6983 - mae: 0.6521 - val_loss: 0.6083 - val_mae: 0.5910
Epoch 2/20
180/180 [==============================] - 2s 14ms/step - loss: 0.4958 - mae: 0.5436 - val_loss: 0.5467 - val_mae: 0.5553
Epoch 3/20
180/180 [==============================] - 3s 14ms/step - loss: 0.4224 - mae: 0.5006 - val_loss: 0.5208 - val_mae: 0.5477
Epoch 4/20
180/180 [==============================] - 3s 14ms/step - loss: 0.3713 - mae: 0.4663 - val_loss: 0.5156 - val_mae: 0.5463
Epoch 5/20
180/180 [==============================] - 2s 14ms/step - loss: 0.3292 - mae: 0.4368 - val_loss: 0.5197 - val_mae: 0.5527
Epoch 6/20
180/180 [==============================] - 3s 14ms/step - loss: 0.2959 - mae: 0.4111 - val_loss: 0.5214 - val_mae: 0.5491
Epoch 7/20
180/180 [==============================] - 3s 15ms/step - loss: 0.2645 - mae: 0.3859 - val_loss: 0.5311 - val_mae: 0.5530
Epoch 8/20
180/180 [==============================] - 2s 13ms/step - 

In [18]:
predictions=model01.predict(X_train)

180/180 [==============================] - 1s 3ms/step
